### rOMERO-gateway Overview
![Overview](romero_overview.png)

### Load rOMERO-gateway

In [ ]:
options(warn=-1)
library(romero.gateway)

### Connect to the server

In [ ]:
user_name = readline('Username: ')
user_password <- getPass::getPass('OMERO password: ')
server <- OMEROServer(host = 'wss://workshop.openmicroscopy.org/omero-ws', port = 443L, username=user_name, password=user_password)
server <- connect(server)
paste('Successfully logged in as', server@user$getUserName())

## Read Data from OMERO

### OMERO dataset and Annotation details
Please fill in the datasetId for the dataset named 'CellProfiler images'

In [ ]:
datasetId <- 1076
fileName <- 'Cells.csv'

### Find the dataset and get the attachment

In [ ]:
omeDataset <- loadObject(server, 'DatasetData', datasetId)
print(paste('Dataset ID:',omeDataset@dataobject$getId()))

# Find the attached csv file
fileAnnotation <- getAnnotations(server, 'DatasetData', getOMEROID(omeDataset), nameFilter = fileName)
annotationFileID = as.integer(fileAnnotation$FileID)
annotationFileID <- annotationFileID[[length(annotationFileID)]]
print(paste('Annotation ID:', annotationFileID))

# Get Image Names
images <- getImages(omeDataset)
image_names <- c()
for (image in images){
    image_names <- c(image_names, image@dataobject$getName())
}

### Load annotation as a R-Dataframe and compare cell areas between the two sets of images

In [ ]:
df <- loadCSV(server, annotationFileID)
dfLength <- subset(df, select=c("ImageNumber", "AreaShape_Area"))
boxplot(dfLength$AreaShape_Area~dfLength$ImageNumber)

In [ ]:
Areas = subset(dfLength, dfLength$ImageNumber == image_names[[1]])$AreaShape_Area
hist(Areas, main = paste("Frequency Histogram of Cellular Areas:",image_names[[1]]))

In [ ]:
Areas = subset(dfLength, dfLength$ImageNumber == image_names[[2]])$AreaShape_Area
hist(Areas, main = paste("Frequency Histogram of Cellular Areas:",image_names[[2]]))

## Save Results back to OMERO

### Attach the plot to an OMERO dataset 

In [ ]:
# Attach the plot to the dataset
tmpfile <- "/tmp/boxplot.png"
png(tmpfile)
boxplot(dfLength$AreaShape_Area~dfLength$ImageNumber)
dev.off()
omero_attach_plot <- attachFile(omeDataset, tmpfile)

print(paste("OMERO File Annotation ID:", omero_attach_plot@dataobject$getId()))

### Attach the t-test output to the dataset

In [ ]:
Areas_2 <- subset(dfLength, dfLength$ImageNumber == image_names[[1]] | dfLength$ImageNumber == image_names[[2]])

t.test(Areas_2$AreaShape_Area~Areas_2$ImageNumber)
tmpfile <- "/tmp/t_test_summary.txt"
cat("Here are my results:\n", file=tmpfile)
capture.output(t.test(Areas_2$AreaShape_Area~Areas_2$ImageNumber), file=tmpfile, append=TRUE)
omero_attach_file <- attachFile(omeDataset, tmpfile)

print(paste("OMERO File Annotation ID:", omero_attach_file@dataobject$getId()))

### Disconnect OMERO session

In [ ]:
server_disconnect <- disconnect(server)

### License (BSD 2-Clause)

Copyright (c) 2021, University of Dundee All rights reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.